In [1]:
import pandas as pd
import numpy as np
import os
import gmplot
from geopy.distance import geodesic
from datetime import datetime
from dateutil import tz

Split up Geo zones for crash data and hard brakes/accelerations
these are the zone categories to use later in the model

In [2]:
GEO_ZONES = [(34.038047, -118.274621),
             (34.047099, -118.268039),
             (34.051926, -118.260303),
             (34.056548, -118.256655),
             (34.061561, -118.250572),
             (34.066321, -118.241543),
             (34.086138, -118.232075),
             (34.094257, -118.242364),
             (34.100011, -118.247934),
             (34.116331, -118.268914)]

In [64]:
# Find a geo zone based on a lat/long

def add_geo_zone(row):
    MAX_DIST_IN_MILES = 0.5
    row_lat = row['latitude']
    row_lon = row['longitude']
    if row_lat and row_lon:
        for idx, coords in enumerate(GEO_ZONES):
            if geodesic(coords, (row_lat,row_lon)).miles <= MAX_DIST_IN_MILES:
                return idx
    return 99

## Looking at LA crash data

In [5]:
la_crash_df = pd.read_csv('./la_open_crash_data.csv')

In [247]:
len(la_crash_df)

499919

In [7]:
la_crash_df.dtypes

DR Number                   int64
Date Reported              object
Date Occurred              object
Time Occurred               int64
Area ID                     int64
Area Name                  object
Reporting District          int64
Crime Code                  int64
Crime Code Description     object
MO Codes                   object
Victim Age                float64
Victim Sex                 object
Victim Descent             object
Premise Code              float64
Premise Description        object
Address                    object
Cross Street               object
Location                   object
dtype: object

In [9]:
# Only use 2016 onwards

def find_2016_2019(row):
    START_DATE = np.datetime64('2016-01-01')
    _date = row['Clean_Date']
    if _date > START_DATE:
        return True
    else:
        return False

In [10]:
la_crash_df['Clean_Date'] = pd.to_datetime(la_crash_df['Date Occurred'])
la_crash_df['in_date_range'] = la_crash_df.apply(lambda row: find_2016_2019(row), axis=1)
crash_df = (la_crash_df[la_crash_df['in_date_range'] == True]).copy()

In [11]:
crash_df.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location,Clean_Date,in_date_range
0,190211527,5/18/19,5/18/19,110,2,Rampart,279,997,TRAFFIC COLLISION,3101 3401 3701 3004 3030,33.0,F,H,101.0,STREET,UNION,JAMES M WOOD,"(34.0508, -118.2731)",2019-05-18,True
2,180618348,8/9/18,8/9/18,1830,6,Hollywood,665,997,TRAFFIC COLLISION,3004 3028 3030 3035 3036 3101 3401 3701 4006,33.0,M,W,101.0,STREET,FRANKLIN AV,ARGYLE,"(34.1052, -118.3252)",2018-08-09,True
5,180626429,12/29/18,12/28/18,1720,6,Hollywood,645,997,TRAFFIC COLLISION,3006 3028 3030 3035 3037 3101 3401 3701 4006,57.0,F,W,108.0,PARKING LOT,HOLLYWOOD BL,HIGHLAND AV,"(34.1016, -118.3387)",2018-12-28,True
6,180213950,6/23/18,6/23/18,1040,2,Rampart,249,997,TRAFFIC COLLISION,3101 3401 3701 3004 3026 3029 3036,39.0,F,H,101.0,STREET,BOYLSTON ST,4TH ST,"(34.0559, -118.2584)",2018-06-23,True
7,180222849,11/16/18,11/15/18,2330,2,Rampart,265,997,TRAFFIC COLLISION,3101 3401 3701 3003 3024 1218,40.0,M,H,101.0,STREET,8TH ST,ALVARADO ST,"(34.0549, -118.2778)",2018-11-15,True


In [12]:
locsplit1 = crash_df['Location'].str.split(",",n=1,expand=True)
clean_lats = locsplit1[0].str.split("(",n=1,expand=True)[1]
clean_lons = locsplit1[1].str.split(")",n=1,expand=True)[0]
type(pd.to_numeric(clean_lats))

pandas.core.series.Series

In [13]:
def clean_up_lat_lon(df):
    locsplit1 = df['Location'].str.split(",",n=1,expand=True)
    clean_lats = locsplit1[0].str.split("(",n=1,expand=True)[1]
    clean_lons = locsplit1[1].str.split(")",n=1,expand=True)[0]
    df['latitude'] = pd.to_numeric(clean_lats)
    df['longitude'] = pd.to_numeric(clean_lons)

In [14]:
clean_up_lat_lon(crash_df)

In [15]:
crash_df['geo_zone'] = crash_df.apply(lambda row: add_geo_zone(row), axis=1)

In [16]:
crash_df['geo_zone'].value_counts()

99    212968
1       1510
2       1325
0       1030
4        657
5        632
3        518
9        371
6        255
7        151
8        131
Name: geo_zone, dtype: int64

Split the day up in 4 time 'categories' as features to use in a model

In [17]:
def create_time_category_for_crash_data(row):
    mil_time = int(row['Time Occurred'])
    if mil_time < 600:
        return 0
    elif mil_time >= 600 and mil_time < 1200:
        return 1
    elif mil_time >= 1200 and mil_time < 1800:
        return 2
    else:
        return 3
    return None

In [18]:
crash_df['time_category'] = crash_df.apply(lambda row: create_time_category_for_crash_data(row), axis=1)
crash_df['time_category'].value_counts()

2    79917
3    61062
1    54996
0    23573
Name: time_category, dtype: int64

# Sirius hard brake / hard accel data

In [20]:
def list_all_files(rootdir):
    full_df = None
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.endswith('.csv'):
#                 print(file_path)
                if full_df is None:
                    full_df = pd.read_csv(file_path)
                else:
                    full_df = pd.concat([full_df, pd.read_csv(file_path)])
                
#             print(os.path.join(subdir, file))
    return full_df

In [21]:
full_hard_events_df = list_all_files('./hard_events/')

In [22]:
len(full_hard_events_df)

17137

In [23]:
full_hard_events_df.head()

,event_type,latitude,longitude,timestamp_ms,g_force,timestamp_utc,vin_trunc,vehicle_id
0,hard_brake,34.257107,-118.606070,1561048165900,0.339905,2019-06-20 16:29:25,1fatp8ff9f5,C_a30de471cabf9d0c
1,hard_brake,34.265840,-118.465996,1561048810900,0.311580,2019-06-20 16:40:10,1fatp8ff9f5,C_a30de471cabf9d0c
2,hard_brake,34.262943,-118.441574,1561049353900,0.368231,2019-06-20 16:49:13,1fatp8ff9f5,C_a30de471cabf9d0c
3,hard_brake,34.259240,-118.437980,1561049408900,0.396556,2019-06-20 16:50:08,1fatp8ff9f5,C_a30de471cabf9d0c
4,hard_brake,34.092068,-118.257810,1561050651900,0.311580,2019-06-20 17:10:51,1fatp8ff9f5,C_a30de471cabf9d0c


In [24]:
hard_brakes = full_hard_events_df[full_hard_events_df['event_type'] == 'hard_brake']
hard_accel = full_hard_events_df[full_hard_events_df['event_type'] == 'hard_accel']
hard_brakes.head()

,event_type,latitude,longitude,timestamp_ms,g_force,timestamp_utc,vin_trunc,vehicle_id
0,hard_brake,34.257107,-118.606070,1561048165900,0.339905,2019-06-20 16:29:25,1fatp8ff9f5,C_a30de471cabf9d0c
1,hard_brake,34.265840,-118.465996,1561048810900,0.311580,2019-06-20 16:40:10,1fatp8ff9f5,C_a30de471cabf9d0c
2,hard_brake,34.262943,-118.441574,1561049353900,0.368231,2019-06-20 16:49:13,1fatp8ff9f5,C_a30de471cabf9d0c
3,hard_brake,34.259240,-118.437980,1561049408900,0.396556,2019-06-20 16:50:08,1fatp8ff9f5,C_a30de471cabf9d0c
4,hard_brake,34.092068,-118.257810,1561050651900,0.311580,2019-06-20 17:10:51,1fatp8ff9f5,C_a30de471cabf9d0c


In [31]:
full_hard_events_df['geo_zone'] = full_hard_events_df.apply(lambda row: add_geo_zone(row), axis=1)
full_hard_events_df['geo_zone'].value_counts()

99    15454
1       684
2       404
0       196
4       123
3       115
5        52
9        45
7        34
8        17
6        13
Name: geo_zone, dtype: int64

In [27]:
def convert_utc_data_to_time_category(row):
    utc_time = row['timestamp_utc']
    from_zone = tz.tzutc()
    to_zone = tz.tzlocal()
    dt = pd.to_datetime(utc_time)
    utc_time_= dt.replace(tzinfo=from_zone)
    pacific_time = utc_time_.astimezone(to_zone)
    pt_hour = int(str(pacific_time.time())[:2])
    return pt_hour

def convert_to_time_category(row):
    pt_hour = row['pt_hour']
    if pt_hour >=0 and pt_hour < 6:
        return 0
    elif pt_hour >=6 and pt_hour <12:
        return 1
    elif pt_hour >= 12 and pt_hour < 18:
        return 2
    else:
        return 3
    return None

In [29]:
full_hard_events_df['pt_hour'] = full_hard_events_df.apply(lambda row: convert_utc_data_to_time_category(row), axis=1)
full_hard_events_df['time_category'] = full_hard_events_df.apply(lambda row: convert_to_time_category(row), axis=1)

In [32]:
# Combining the two and making negative samples
all_incidents = full_hard_events_df[['latitude','longitude','geo_zone','time_category','event_type']].copy()
crash_df['event_type'] = 'crash'
all_incidents = pd.concat([all_incidents,crash_df[['latitude','longitude','geo_zone','time_category','event_type']]])

In [33]:
all_incidents['slowdown'] = 1
all_relevant_incidents = all_incidents[all_incidents['geo_zone'] < 99]
all_relevant_incidents.sample()

,latitude,longitude,geo_zone,time_category,event_type,slowdown
46,34.05577,-118.25676,2,3,hard_accel,1


In [34]:
all_relevant_incidents['COUNTER'] = 1

/Users/ashwinkumar/dl_venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
groupdata = all_relevant_incidents.groupby(['time_category','geo_zone'])['COUNTER'].sum()

def find_incident_count(row):
    return groupdata[row['time_category']][row['geo_zone']]

def find_incident_quartile(row, incident_counts):
    thresholds = np.percentile(incident_counts, [25,50,75])
    if row['group_incident_count'] < thresholds[0]:
        return 0
    elif row['group_incident_count'] < thresholds[1]:
        return 1
    elif row['group_incident_count'] < thresholds[2]:
        return 2
    else:
        return 3
    

In [36]:
all_relevant_incidents['group_incident_count'] = all_relevant_incidents.apply(lambda row: find_incident_count(row), axis=1)

/Users/ashwinkumar/dl_venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
all_relevant_incidents.sample(10)

,latitude,longitude,geo_zone,time_category,event_type,slowdown,COUNTER,group_incident_count
15275,34.11410,-118.27550,9,3,crash,1,1,113
103,34.04588,-118.26312,1,2,hard_accel,1,1,810
425235,34.04870,-118.26270,1,3,crash,1,1,681
370619,34.06820,-118.25020,4,2,crash,1,1,318
316122,34.06770,-118.25200,4,2,crash,1,1,318
19777,34.04590,-118.26690,1,0,crash,1,1,233
11838,34.06520,-118.24920,4,1,crash,1,1,205
395580,34.09120,-118.23910,7,1,crash,1,1,56
312594,34.05190,-118.25310,2,1,crash,1,1,446
371929,34.04390,-118.27090,0,1,crash,1,1,316


In [39]:
incident_counts = sorted(list(set(all_relevant_incidents['group_incident_count'])))
all_relevant_incidents['incident_quartile'] = all_relevant_incidents.apply(lambda row: find_incident_quartile(row, incident_counts), axis=1)

/Users/ashwinkumar/dl_venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [40]:
all_relevant_incidents.head()

,latitude,longitude,geo_zone,time_category,event_type,slowdown,COUNTER,group_incident_count,incident_quartile
12,34.058290,-118.255800,3,1,hard_brake,1,1,175,2
13,34.053127,-118.258720,2,1,hard_brake,1,1,446,3
14,34.034603,-118.268074,0,1,hard_brake,1,1,316,3
17,34.036964,-118.270220,0,1,hard_brake,1,1,316,3
18,34.038560,-118.271900,0,1,hard_brake,1,1,316,3


In [41]:
# Use a random forest to figure out incident quartile based on geo zone of the route and the time of day

In [42]:
from sklearn.ensemble import RandomForestClassifier
model_1_df = all_relevant_incidents.copy()
model_1_df.dtypes

latitude                float64
longitude               float64
geo_zone                  int64
time_category             int64
event_type               object
slowdown                  int64
COUNTER                   int64
group_incident_count      int64
incident_quartile         int64
dtype: object

In [43]:
model_1_df['incident_quartile'] = pd.Categorical(model_1_df.incident_quartile)

In [44]:
len(model_1_df)

8263

In [45]:
from sklearn.model_selection import train_test_split 

In [46]:
model_1_X = model_1_df[['geo_zone','time_category']].copy()
model_1_y = model_1_df['incident_quartile']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(model_1_X, model_1_y, test_size=0.2, random_state=0)

In [48]:
clf = RandomForestClassifier(n_jobs=2, random_state=0)
clf.fit(X_train, y_train)

/Users/ashwinkumar/dl_venv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [49]:
clf.predict(X_test)

array([3, 1, 2, ..., 3, 3, 3])

In [50]:
list(zip(X_train, clf.feature_importances_))

[('geo_zone', 0.7786589410674416), ('time_category', 0.22134105893255834)]

In [51]:
model_1_preds = clf.predict(X_test)

In [52]:
pd.crosstab(y_test,model_1_preds,rownames=['Actual Incident Quartile'],colnames=['Predicted Incident Quartile'])

Predicted Incident Quartile,0,1,2,3
Actual Incident Quartile,,,,
0,95,0,0,0
1,0,214,0,0
2,0,0,365,0
3,0,0,0,979


In [53]:
X_test.shape

(1653, 2)

In [54]:
X_test.values

array([[2, 2],
       [3, 3],
       [3, 2],
       ...,
       [1, 3],
       [1, 3],
       [0, 2]])

In [55]:
new_data = np.array([3,3]).reshape(1,-1)

In [56]:
clf.predict(new_data)

array([1])

In [57]:
def predict_incident_quartile(clf, route_data):
    # Route data needs to be array to be in format [ (lat,lon,timecategory)]
    # Convert route data into X
        # Convert lat/lon 
    rdf = pd.DataFrame(route_data, columns=['latitude','longitude','time_category'])
    rdf['geo_zone'] = rdf.apply(lambda row: add_geo_zone(row), axis=1)
    rdf_X = rdf[['geo_zone','time_category']].copy()
    rdf_preds = clf.predict(rdf_X)
    predicted_incident_quartile_avg = rdf_preds.mean()
    return predicted_incident_quartile_avg

In [58]:
route_data = [(34.038047, -118.274621, 2),
             (34.047099, -118.268039, 2),
             (34.051926, -118.260303, 2),
             (34.056548, -118.256655, 2),
             (34.061561, -118.250572, 2),
             (34.066321, -118.241543, 2),
             (34.086138, -118.232075, 2),
             (34.094257, -118.242364, 2),
             (34.100011, -118.247934, 2),
             (34.116331, -118.268914, 2)]

In [59]:
predict_incident_quartile(clf, route_data)

1.9

# Plot map data

In [60]:
gmap = gmplot.GoogleMapPlotter(35, -102, 5)
# gmap.scatter(hard_events_latitudes[:1000], hard_events_longitudes[:1000], 'red', size = 10)
gmap.scatter(hard_brakes.loc[:,'latitude'][:1000], 
             hard_brakes.loc[:,'longitude'][:1000], 
             'red', 
             size = 8)
gmap.scatter(hard_accel.loc[:,'latitude'][:1000], 
             hard_accel.loc[:,'longitude'][:1000], 
             'green', 
             size = 8)
gmap.scatter(crash_df.loc[:,'latitude'][:1000], 
             crash_df.loc[:,'longitude'][:1000], 
             'yellow', 
             size = 8)
gmap.draw('gmplot.html')

In [61]:
full_hard_events_df.head()

,event_type,latitude,longitude,timestamp_ms,g_force,timestamp_utc,vin_trunc,vehicle_id,pt_hour,time_category,geo_zone
0,hard_brake,34.257107,-118.606070,1561048165900,0.339905,2019-06-20 16:29:25,1fatp8ff9f5,C_a30de471cabf9d0c,9,1,99
1,hard_brake,34.265840,-118.465996,1561048810900,0.311580,2019-06-20 16:40:10,1fatp8ff9f5,C_a30de471cabf9d0c,9,1,99
2,hard_brake,34.262943,-118.441574,1561049353900,0.368231,2019-06-20 16:49:13,1fatp8ff9f5,C_a30de471cabf9d0c,9,1,99
3,hard_brake,34.259240,-118.437980,1561049408900,0.396556,2019-06-20 16:50:08,1fatp8ff9f5,C_a30de471cabf9d0c,9,1,99
4,hard_brake,34.092068,-118.257810,1561050651900,0.311580,2019-06-20 17:10:51,1fatp8ff9f5,C_a30de471cabf9d0c,10,1,99


# Fuel Efficiency

In [62]:
fuel_eff_df = list_all_files('./fuel_data_siriusxm/')

In [ ]:
# Use only one car, one of the ones with more data
fuel_eff_df.groupby('vehicle_id').count().sort_values(by='fuel_level', ascending=False)[:5]

In [ ]:
car_df = fuel_eff_df[fuel_eff_df['vehicle_id'] == 'C_f42acfa01b52c500'].copy()
car_df.sort_values(by='timestamp_ms', inplace=True)

In [179]:
# Chevy appropriate data
chevy_df = pd.read_csv('./chevy_data_fuel.csv')

In [180]:
chevy_df['timestamp_utc'] = pd.to_datetime(chevy_df['timestamp_utc'])
chevy_minute_df = chevy_df.resample('1T', on='timestamp_utc').mean()
chevy_minute_df.tail()

,Unnamed: 0,accel_x,accel_y,accel_z,rpm,speed_kmph,maf,map,iat,eng_load,eng_coolant_temp,fuel_level,on_time,timestamp_ms
timestamp_utc,,,,,,,,,,,,,,
2019-06-06 17:45:00,40214.666667,NaN,NaN,NaN,545.166667,0.666667,542.166667,NaN,37.0,45.333333,133.0,40.666667,12081.000000,1.560000e+12
2019-06-06 17:46:00,41150.333333,NaN,NaN,NaN,1322.166667,42.333333,3062.166667,NaN,38.0,127.166667,130.0,29.333333,12141.000000,1.560000e+12
2019-06-06 17:47:00,42020.833333,NaN,NaN,NaN,1193.666667,65.666667,1888.833333,NaN,34.0,100.333333,130.0,34.333333,12201.000000,1.560000e+12
2019-06-06 17:48:00,42906.500000,NaN,NaN,NaN,1234.166667,68.500000,2900.666667,NaN,33.0,145.666667,130.0,32.166667,12261.000000,1.560000e+12
2019-06-06 17:49:00,43838.333333,NaN,NaN,NaN,817.833333,21.833333,756.666667,NaN,33.0,45.333333,132.0,23.666667,12319.666667,1.560000e+12


In [181]:
# Calcualte difference in fuel readings at each minute
chevy_minute_df['fuel_diff'] = chevy_minute_df['fuel_level'] - chevy_minute_df['fuel_level'].shift(1)

In [70]:
# Add in the data from AWS Connected Cars
import boto3
s3_resource = boto3.resource('s3')
s3_client = boto3.client('s3')

import uuid
def create_bucket_name(bucket_prefix):
    # The generated bucket name must be between 3 and 63 chars long
    return ''.join([bucket_prefix, str(uuid.uuid4())])

In [71]:
s3_client.list_buckets()

{'ResponseMetadata': {'RequestId': '38BCC3038CCB68FC',
  'HostId': '1qzsSBxzY6GPtJafbVDOZNsOa0DsbuEX7id7Fp2jgHwx/y3esuCJYp+6VyYOsAmVL4rM7YN++jo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1qzsSBxzY6GPtJafbVDOZNsOa0DsbuEX7id7Fp2jgHwx/y3esuCJYp+6VyYOsAmVL4rM7YN++jo=',
   'x-amz-request-id': '38BCC3038CCB68FC',
   'date': 'Mon, 18 Nov 2019 04:34:13 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'ashwinawss3bucket',
   'CreationDate': datetime.datetime(2019, 10, 26, 0, 26, 16, tzinfo=tzutc())},
  {'Name': 'connected-vehicle-data-us-east-1-869647407607',
   'CreationDate': datetime.datetime(2019, 11, 7, 0, 18, 51, tzinfo=tzutc())},
  {'Name': 'connected-vehicle-trip-us-east-1-869647407607',
   'CreationDate': datetime.datetime(2019, 11, 7, 0, 18, 50, tzinfo=tzutc())},
  {'Name': 'elasticbeanstalk-us-west-1-869647407607',
   'CreationDate': datetime.datetime(2014, 8, 17, 23

In [73]:
# bucket_name = "ashwinawss3bucket/HackHLTH_Oct2019/practicing"
bucket_name = "ashwinawss3bucket"
# s3_client.download_file(bucket_name, '/HackHLTH_Oct2019/practicing/application.py', 'dlapplication.py')
s3_client.list_objects(Bucket=bucket_name)['Contents']
my_key = "telemetry/2019/11/07/00/connected-vehicle-telemetry-1-2019-11-07-00-51-52-ed786c56-a6f6-43de-a1b8-c860d6bb0472.gz"
my_bucket_name = "connected-vehicle-data-us-east-1-869647407607"
s3_client.download_file(Bucket=my_bucket_name, Key=my_key, Filename='mydata.gz')

In [74]:
import gzip
import shutil
with gzip.open('mydata.gz', 'rb') as f_in:
    with open('mydata.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [76]:
aws_connected_df = pd.read_json('mydata.json', orient='columns', lines=True)

In [77]:
aws_connected_df.head()

,latitude,longitude,name,timestamp,trip_id,value,vin
0,NaN,NaN,ignition_status,2019-11-07 00:51:52.825,10be1006-cba7-4c89-878d-658676c974ca,run,PUTR68I043XT1CEUZ
1,NaN,NaN,fuel_level,2019-11-07 00:51:52.825,10be1006-cba7-4c89-878d-658676c974ca,100,PUTR68I043XT1CEUZ
2,NaN,NaN,transmission_gear_position,2019-11-07 00:51:52.825,10be1006-cba7-4c89-878d-658676c974ca,first,PUTR68I043XT1CEUZ
3,NaN,NaN,brake,2019-11-07 00:51:52.826,10be1006-cba7-4c89-878d-658676c974ca,0,PUTR68I043XT1CEUZ
4,NaN,NaN,steering_wheel_angle,2019-11-07 00:51:52.821,10be1006-cba7-4c89-878d-658676c974ca,0,PUTR68I043XT1CEUZ


In [78]:
import json

the_objects = []
for line in open('mydata.json', 'r'):
    the_objects.append(json.loads(line))

In [79]:
the_objects

[{'timestamp': '2019-11-07 00:51:52.825000000',
  'trip_id': '10be1006-cba7-4c89-878d-658676c974ca',
  'vin': 'PUTR68I043XT1CEUZ',
  'name': 'ignition_status',
  'value': 'run'},
 {'timestamp': '2019-11-07 00:51:52.825000000',
  'trip_id': '10be1006-cba7-4c89-878d-658676c974ca',
  'vin': 'PUTR68I043XT1CEUZ',
  'name': 'fuel_level',
  'value': 100},
 {'timestamp': '2019-11-07 00:51:52.825000000',
  'trip_id': '10be1006-cba7-4c89-878d-658676c974ca',
  'vin': 'PUTR68I043XT1CEUZ',
  'name': 'transmission_gear_position',
  'value': 'first'},
 {'timestamp': '2019-11-07 00:51:52.826000000',
  'trip_id': '10be1006-cba7-4c89-878d-658676c974ca',
  'vin': 'PUTR68I043XT1CEUZ',
  'name': 'brake',
  'value': 0},
 {'timestamp': '2019-11-07 00:51:52.821000000',
  'trip_id': '10be1006-cba7-4c89-878d-658676c974ca',
  'vin': 'PUTR68I043XT1CEUZ',
  'name': 'steering_wheel_angle',
  'value': 0},
 {'timestamp': '2019-11-07 00:51:52.824000000',
  'trip_id': '10be1006-cba7-4c89-878d-658676c974ca',
  'vin': 'P

In [80]:
list(set(aws_connected_df['name'].values))

['brake',
 'brake_pedal_status',
 'ignition_status',
 'steering_wheel_angle',
 'fuel_consumed_since_restart',
 'parking_brake_status',
 'location',
 'vehicle_speed',
 'accelerator_pedal_position',
 'gear_lever_position',
 'transmission_gear_position',
 'acceleration',
 'engine_speed',
 'odometer',
 'oil_temp',
 'torque_at_transmission',
 'fuel_level']

In [89]:
# We need to get the columns to be that information
# and the row index will be the timestamp
df_dict = {}
for o in the_objects:
    ts = o['timestamp']
    if o['name'] != 'location':
        if ts in df_dict:
            df_dict[ts][o['name']] = o['value']
        else:
            df_dict[ts] = {}
            df_dict[ts][o['name']] = o['value']
    



In [90]:
data = {'col_1': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}

{'2019-11-07 00:51:52.825000000': {'ignition_status': 'run',
  'fuel_level': 100,
  'transmission_gear_position': 'first',
  'odometer': 0.001,
  'gear_lever_position': 'drive',
  'fuel_consumed_since_restart': 0.000524},
 '2019-11-07 00:51:52.826000000': {'brake': 0, 'oil_temp': 3.7},
 '2019-11-07 00:51:52.821000000': {'steering_wheel_angle': 0},
 '2019-11-07 00:51:52.824000000': {'acceleration': 0,
  'parking_brake_status': False,
  'accelerator_pedal_position': 31.8,
  'brake_pedal_status': False},
 '2019-11-07 00:51:52.923000000': {'parking_brake_status': False,
  'ignition_status': 'run',
  'accelerator_pedal_position': 33.2,
  'transmission_gear_position': 'first',
  'brake_pedal_status': False,
  'odometer': 0.001,
  'fuel_level': 100,
  'gear_lever_position': 'drive'},
 '2019-11-07 00:51:52.823000000': {'engine_speed': 1563.8,
  'vehicle_speed': 19.07,
  'torque_at_transmission': 429.2},
 '2019-11-07 00:51:52.922000000': {'engine_speed': 1631.1,
  'vehicle_speed': 19.91,
  'acc

In [95]:
aws_connected_df = pd.DataFrame.from_dict(df_dict, orient='index')

In [98]:
aws_fuel_only = aws_connected_df[aws_connected_df['fuel_level'] > 0]

In [99]:
aws_fuel_only.head()

,ignition_status,fuel_level,transmission_gear_position,odometer,gear_lever_position,fuel_consumed_since_restart,brake,oil_temp,steering_wheel_angle,acceleration,parking_brake_status,accelerator_pedal_position,brake_pedal_status,engine_speed,vehicle_speed,torque_at_transmission
2019-11-07 00:51:52.825000000,run,100.00,first,0.001,drive,0.000524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.923000000,run,100.00,first,0.001,drive,NaN,NaN,NaN,NaN,NaN,False,33.2,False,NaN,NaN,NaN
2019-11-07 00:51:54.844000000,NaN,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.940000000,NaN,100.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:56.864000000,NaN,99.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [100]:
aws_connected_df.head(100)

,ignition_status,fuel_level,transmission_gear_position,odometer,gear_lever_position,fuel_consumed_since_restart,brake,oil_temp,steering_wheel_angle,acceleration,parking_brake_status,accelerator_pedal_position,brake_pedal_status,engine_speed,vehicle_speed,torque_at_transmission
2019-11-07 00:51:52.821000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.823000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1563.8,19.07,429.2
2019-11-07 00:51:52.824000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,False,31.8,False,NaN,NaN,NaN
2019-11-07 00:51:52.825000000,run,100.00,first,0.001,drive,0.000524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.826000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.920000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.922000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,1631.1,19.91,448.5
2019-11-07 00:51:52.923000000,run,100.00,first,0.001,drive,NaN,NaN,NaN,NaN,NaN,False,33.2,False,NaN,NaN,NaN
2019-11-07 00:51:52.924000000,NaN,NaN,NaN,NaN,NaN,0.000545,0.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.830000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
aws_connected_df[aws_connected_df['fuel_consumed_since_restart'] > 0]

,ignition_status,fuel_level,transmission_gear_position,odometer,gear_lever_position,fuel_consumed_since_restart,brake,oil_temp,steering_wheel_angle,acceleration,parking_brake_status,accelerator_pedal_position,brake_pedal_status,engine_speed,vehicle_speed,torque_at_transmission
2019-11-07 00:51:52.825000000,run,100.00,first,0.001,drive,0.000524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.924000000,NaN,NaN,NaN,NaN,NaN,0.000545,0.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.845000000,NaN,NaN,NaN,NaN,NaN,0.001540,NaN,7.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.941000000,NaN,NaN,NaN,NaN,NaN,0.001605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:56.865000000,NaN,NaN,NaN,NaN,NaN,0.002556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:56.958000000,NaN,99.99,NaN,NaN,NaN,0.002664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:58.894000000,NaN,NaN,NaN,NaN,NaN,0.003588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:58.987000000,NaN,NaN,NaN,NaN,NaN,0.003693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:52:00.914000000,NaN,NaN,NaN,NaN,NaN,0.004665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:52:01.005000000,NaN,NaN,NaN,NaN,NaN,0.004631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
aws_connected_df.count(axis='rows')

ignition_status                298
fuel_level                     298
transmission_gear_position     298
odometer                       298
gear_lever_position            298
fuel_consumed_since_restart    298
brake                          298
oil_temp                       298
steering_wheel_angle           298
acceleration                   298
parking_brake_status           298
accelerator_pedal_position     298
brake_pedal_status             298
engine_speed                   298
vehicle_speed                  298
torque_at_transmission         298
dtype: int64

In [104]:
len(aws_connected_df)

3994

In [ ]:
m = df.iloc[:, 0].eq('T')
cumgrp = m.cumsum()[~m]
grps = df[~m].groupby(cumgrp)

In [105]:
aws_connected_df.head(100)

,ignition_status,fuel_level,transmission_gear_position,odometer,gear_lever_position,fuel_consumed_since_restart,brake,oil_temp,steering_wheel_angle,acceleration,parking_brake_status,accelerator_pedal_position,brake_pedal_status,engine_speed,vehicle_speed,torque_at_transmission
2019-11-07 00:51:52.821000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.823000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1563.8,19.07,429.2
2019-11-07 00:51:52.824000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,False,31.8,False,NaN,NaN,NaN
2019-11-07 00:51:52.825000000,run,100.00,first,0.001,drive,0.000524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.826000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.920000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.922000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,1631.1,19.91,448.5
2019-11-07 00:51:52.923000000,run,100.00,first,0.001,drive,NaN,NaN,NaN,NaN,NaN,False,33.2,False,NaN,NaN,NaN
2019-11-07 00:51:52.924000000,NaN,NaN,NaN,NaN,NaN,0.000545,0.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.830000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
m = aws_connected_df.iloc[:, 5].ge(0.0)
# cumgrp = m.cumsum()[~m]
# grps = df[~m].groupby(cumgrp)

In [109]:
cumgrp = m.cumsum()[~m]

In [110]:
grps = aws_connected_df[~m].groupby(cumgrp)

In [112]:
list_of_groups = [g for n, g in grps]

In [117]:
len(list_of_groups)

299

In [125]:
fuel_used_rows = aws_connected_df[m]

In [127]:
fuel_used_rows

,ignition_status,fuel_level,transmission_gear_position,odometer,gear_lever_position,fuel_consumed_since_restart,brake,oil_temp,steering_wheel_angle,acceleration,parking_brake_status,accelerator_pedal_position,brake_pedal_status,engine_speed,vehicle_speed,torque_at_transmission
2019-11-07 00:51:52.825000000,run,100.00,first,0.001,drive,0.000524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:52.924000000,NaN,NaN,NaN,NaN,NaN,0.000545,0.0,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.845000000,NaN,NaN,NaN,NaN,NaN,0.001540,NaN,7.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.941000000,NaN,NaN,NaN,NaN,NaN,0.001605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:56.865000000,NaN,NaN,NaN,NaN,NaN,0.002556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:56.958000000,NaN,99.99,NaN,NaN,NaN,0.002664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:58.894000000,NaN,NaN,NaN,NaN,NaN,0.003588,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:58.987000000,NaN,NaN,NaN,NaN,NaN,0.003693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:52:00.914000000,NaN,NaN,NaN,NaN,NaN,0.004665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:52:01.005000000,NaN,NaN,NaN,NaN,NaN,0.004631,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
list_of_groups[3]

,ignition_status,fuel_level,transmission_gear_position,odometer,gear_lever_position,fuel_consumed_since_restart,brake,oil_temp,steering_wheel_angle,acceleration,parking_brake_status,accelerator_pedal_position,brake_pedal_status,engine_speed,vehicle_speed,torque_at_transmission
2019-11-07 00:51:54.847000000,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.926000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.928000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,351.8
2019-11-07 00:51:54.929000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3475.5,NaN,NaN
2019-11-07 00:51:54.930000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46.43,NaN
2019-11-07 00:51:54.931000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.7773,NaN,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.932000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.2,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.933000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
2019-11-07 00:51:54.934000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN
2019-11-07 00:51:54.935000000,NaN,NaN,second,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# engine_speed
# vehicle_speed
# torque_at_transmission
# acceleration

In [207]:
all_dicts = list()
for i in range(len(list_of_groups)-1):
    engine_speed_avg = list_of_groups[i]['engine_speed'].mean()
    vehicle_speed_avg = list_of_groups[i]['vehicle_speed'].mean()
    torque_at_transmission_avg = list_of_groups[i]['torque_at_transmission'].mean()
    acceleration_avg = list_of_groups[i]['vehicle_speed'].mean()
    fuel_used = fuel_used_rows.iloc[i,5]
    group_dict = {
        'engine_speed_avg': engine_speed_avg,
        'vehicle_speed_avg': vehicle_speed_avg,
        'torque_at_transmission_avg': torque_at_transmission_avg,
        'acceleration_avg': acceleration_avg,
        'fuel_used': fuel_used
    }
    all_dicts.append(group_dict)

In [217]:
aws_usable_df = pd.DataFrame(all_dicts)

In [218]:
aws_usable_df.head()

,acceleration_avg,engine_speed_avg,fuel_used,torque_at_transmission_avg,vehicle_speed_avg
0,19.07,1563.8,0.000524,429.2,19.07
1,19.91,1631.1,0.000545,448.5,19.91
2,44.48,3328.2,0.001540,336.7,44.48
3,46.43,3475.5,0.001605,351.8,46.43
4,56.73,3009.9,0.002556,299.9,56.73


In [132]:
fuel_used_rows.iloc[0,5]

0.000524

In [210]:
len(aws_usable_df)

298

In [219]:
for i in range(10):
    aws_usable_df = aws_usable_df.append(aws_usable_df)

In [220]:
len(aws_usable_df)

305152

In [147]:
# Simulating historical data
aws_usable_df = aws_usable_df.append(aws_usable_df*1000)

In [221]:
len(aws_usable_df)

305152

In [159]:
chevy_minute_df = chevy_minute_df.append(chevy_minute_df*1000, ignore_index=True)

In [160]:
len(chevy_minute_df)

2264

The chevy model (SiriusXM data)

In [178]:
chevy_minute_df.head(10)

,Unnamed: 0,accel_x,accel_y,accel_z,rpm,speed_kmph,maf,map,iat,eng_load,eng_coolant_temp,fuel_level,on_time,timestamp_ms,fuel_diff
0,1510.000000,NaN,NaN,NaN,984.000000,14.142857,1493.571429,NaN,39.0,89.000000,139.0,112.857143,26.285714,1.560000e+12,NaN
1,2146.666667,NaN,NaN,NaN,708.666667,2.500000,1384.333333,NaN,34.0,88.833333,132.0,111.000000,90.000000,1.560000e+12,-1.857143
2,2668.000000,NaN,NaN,NaN,1072.166667,24.500000,2930.166667,NaN,33.0,134.833333,132.0,121.833333,150.333333,1.560000e+12,10.833333
3,3148.000000,NaN,NaN,NaN,1182.333333,60.000000,2386.333333,NaN,33.0,115.833333,131.0,112.666667,211.000000,1.560000e+12,-9.166667
4,3628.000000,NaN,NaN,NaN,959.833333,35.833333,1268.833333,NaN,32.0,73.666667,133.0,111.500000,271.000000,1.560000e+12,-1.166667
5,4108.000000,NaN,NaN,NaN,1332.833333,36.666667,3346.500000,NaN,33.0,142.833333,133.0,114.000000,331.000000,1.560000e+12,2.500000
6,4587.000000,NaN,NaN,NaN,780.166667,13.500000,1327.166667,NaN,33.0,79.833333,133.0,117.500000,391.000000,1.560000e+12,3.500000
7,5066.000000,NaN,NaN,NaN,1017.500000,22.166667,2173.000000,NaN,34.0,84.500000,134.0,111.833333,451.000000,1.560000e+12,-5.666667
8,5527.666667,NaN,NaN,NaN,989.000000,21.166667,1445.166667,NaN,34.0,87.833333,133.0,109.333333,511.000000,1.560000e+12,-2.500000
9,5929.000000,NaN,NaN,NaN,1187.000000,32.500000,2361.833333,NaN,35.0,99.833333,134.0,123.833333,571.000000,1.560000e+12,14.500000


In [162]:
from sklearn import preprocessing

In [182]:
chevy_rel_df = chevy_minute_df[['rpm','speed_kmph','maf','iat','eng_load','eng_coolant_temp','fuel_diff']].copy()
chevy_rel_df.dropna(inplace=True)
X_chevy = chevy_rel_df[['rpm','speed_kmph','maf','iat','eng_load','eng_coolant_temp']].copy()
scaler = preprocessing.StandardScaler().fit(X_chevy)
X_scaled = scaler.transform(X_chevy)
y_chevy = chevy_rel_df['fuel_diff'].values

In [183]:
len(X_chevy), len(y_chevy)

(240, 240)

In [184]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [186]:
X_chevy

,rpm,speed_kmph,maf,iat,eng_load,eng_coolant_temp
timestamp_utc,,,,,,
2019-06-06 13:08:00,708.666667,2.500000,1384.333333,34.0,88.833333,132.0
2019-06-06 13:09:00,1072.166667,24.500000,2930.166667,33.0,134.833333,132.0
2019-06-06 13:10:00,1182.333333,60.000000,2386.333333,33.0,115.833333,131.0
2019-06-06 13:11:00,959.833333,35.833333,1268.833333,32.0,73.666667,133.0
2019-06-06 13:12:00,1332.833333,36.666667,3346.500000,33.0,142.833333,133.0
2019-06-06 13:13:00,780.166667,13.500000,1327.166667,33.0,79.833333,133.0
2019-06-06 13:14:00,1017.500000,22.166667,2173.000000,34.0,84.500000,134.0
2019-06-06 13:15:00,989.000000,21.166667,1445.166667,34.0,87.833333,133.0
2019-06-06 13:16:00,1187.000000,32.500000,2361.833333,35.0,99.833333,134.0


In [187]:
sirius_regressor = LinearRegression()
sirius_regressor.fit(X_scaled, y_chevy)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [188]:
sirius_regressor.coef_

array([-6.38847214,  0.90387147,  6.33111328, -0.46648558, -1.89269634,
       -0.6320034 ])

In [189]:
y_pred_sirius = sirius_regressor.predict(X_scaled)

In [190]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_chevy, y_pred_sirius))  
print('Mean Squared Error:', metrics.mean_squared_error(y_chevy, y_pred_sirius))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_chevy, y_pred_sirius)))

Mean Absolute Error: 6.9134424055622
Mean Squared Error: 88.38320907093612
Root Mean Squared Error: 9.401234443993838


In [234]:
new_x_sirius = X_chevy.iloc[0].values.reshape(1,-1)
new_x_sirius = scaler.transform(new_x_sirius)
sirius_regressor.predict(new_x_sirius)

array([3.54531321])

AWS Regressor

In [222]:
aws_usable_df.dropna(inplace=True)
X_aws = aws_usable_df[['acceleration_avg', 'engine_speed_avg',
       'torque_at_transmission_avg', 'vehicle_speed_avg']].copy()
aws_scaler = preprocessing.StandardScaler().fit(X_aws)
X_aws_scaled = aws_scaler.transform(X_aws)
y_aws = aws_usable_df['fuel_used'].values

In [223]:
aws_regressor = LinearRegression()
aws_regressor.fit(X_aws_scaled, y_aws)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [224]:
aws_regressor.coef_

array([ 1.77857829e+09, -8.46191917e-03, -4.49425602e-03, -1.77857829e+09])

In [235]:
new_x_aws = X_aws.iloc[0].values.reshape(1,-1)
new_x_aws = aws_scaler.transform(new_x_aws)
aws_regressor.predict(new_x_aws)

array([0.04201958])

# Export models


In [230]:
import pickle
pickle.dump(aws_regressor, open('aws_model.sav','wb'))
pickle.dump(aws_scaler, open('aws_scaler.sav','wb'))
pickle.dump(sirius_regressor, open('sirius_fuel_eff_model.sav','wb'))
pickle.dump(scaler, open('sirius_fuel_eff_scaler.sav', 'wb'))
pickle.dump(clf, open('incident_model.sav','wb'))

In [237]:
aws_fuel_used = aws_regressor.predict(new_x_aws)

In [238]:
sirius_fuel_used = sirius_regressor.predict(new_x_sirius)

In [240]:
total_fuel_usage = aws_fuel_used[0] + sirius_fuel_used[0]

3.5873327905650663

In [242]:
fuel_eff_x = np.array([ 708.66666667,2.5,1384.33333333,34.0,88.83333333,132.0]).reshape(1,-1)
sirius_data_x = scaler.transform(fuel_eff_x)
sirius_regressor.predict(sirius_data_x)

array([3.54531321])

In [243]:
total_fuel_usage_pred = aws_fuel_used[0] + sirius_fuel_used[0]

In [244]:
incident_pred = predict_incident_quartile(clf, route_data)

In [246]:
total_fuel_usage_pred

3.5873327905650663

In [ ]:
if total_fuel_usage_pred > 3.0 and total_fuel_usage_pred < 6.0:
    if incident_pred > 1.0 and incident_pred < 2.0:
        NUM_STOPS = 2
    elif incident_pred < 1.0:
        NUM_STOPS = 1
    elif incident_pred > 2.0:
        NUM_STOPS = 3
elif total_fuel_usage_pred < 3.0:
    NUM_STOPS = 1
else:
    NUM_STOPS = 4